In [21]:
import json

# The issue is that json.loads() expects a JSON string, not a file path
# We need to open the file and read its contents first
with open("data/sources.json", "r") as file:
    data = json.load(file)  # json.load() reads directly from a file object
    
data

[{'id': '1',
  'name': 'total private construction',
  'varname': 'res',
  'fredcode': 'TLPRVCONS',
  'link': 'https://fred.stlouisfed.org/series/',
  'source': 'U. S. Census Bureau',
  'release': 'construction spending',
  'frequency': 'monthly',
  'units': 'million of dollars, seasonally adjusted annual rate',
  'tables': None,
  'description': 'Monthly series for total private construction spending.',
  'start_date': '1993-01-31',
  'end_date': '9999-99-99',
  'last_updated': '9999',
  'notes': 'Data from U. S. Census Bureau.'},
 {'id': '2',
  'name': 'total private residential construction',
  'varname': 'priv',
  'fredcode': 'PRRESCONS',
  'link': 'https://fred.stlouisfed.org/series/',
  'source': 'U. S. Census Bureau',
  'release': 'construction spending',
  'frequency': 'monthly',
  'units': 'million of dollars, seasonally adjusted annual rate',
  'tables': None,
  'description': 'Monthly series for total private residential construction spending.',
  'start_date': '1993-01-31',

In [22]:
import requests
from interpolation_sw_2010.aws_manager.s3 import S3

def get_table_2_7A(api_key):
    """
    Downloads NIPA Table 2.7A (Wages and Salaries by Industry, Monthly).
    """
    url = "https://apps.bea.gov/api/data"
    params = {
        "UserID": api_key,
        "method": "GetData",
        "datasetname": "NIPA",
        # Table 2.7A is typically identified as "T20700" in the NIPA parameter list
        "TableName": "T20700",  
        "Frequency": "M",       # Monthly
        "Year": "ALL",          # Or specify e.g. "2020,2021" if you want fewer years
        "ResultFormat": "JSON"  # or "XML"
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

if __name__ == "__main__":
    api_key = json.loads(S3.get_secret("BEA-API-KEY"))["api_key"]
    data = get_table_2_7A(api_key)
    # Do something with the returned JSON
    print(data)

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'USERID', 'ParameterValue': 'D6C458B8-0AAF-484F-91AB-E298E5DC61D7'}, {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATA'}, {'ParameterName': 'DATASETNAME', 'ParameterValue': 'NIPA'}, {'ParameterName': 'TABLENAME', 'ParameterValue': 'T20700'}, {'ParameterName': 'FREQUENCY', 'ParameterValue': 'M'}, {'ParameterName': 'YEAR', 'ParameterValue': 'ALL'}, {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'}, {'ParameterName': 'ShowMillions', 'ParameterValue': 'N'}]}, 'Error': {'APIErrorDescription': 'Error retrieving NIPA data.', 'APIErrorCode': '201', 'ErrorDetail': {'Description': 'Invalid TableName was requested.'}}}}


In [35]:

import beapy
bea = beapy.BEA(key=api_key)
res = bea.parameter_list("national")


KeyError: "no recognized dataset with name 'national'"

In [31]:
res.datasets

{'NIPA': 'Standard NIPA tables',
 'NIUnderlyingDetail': 'Standard NI underlying detail tables',
 'MNE': 'Multinational Enterprises',
 'FixedAssets': 'Standard Fixed Assets tables',
 'ITA': 'International Transactions Accounts',
 'IIP': 'International Investment Position',
 'InputOutput': 'Input-Output Data',
 'IntlServTrade': 'International Services Trade',
 'IntlServSTA': 'International Services Supplied Through Affiliates',
 'GDPbyIndustry': 'GDP by Industry',
 'Regional': 'Regional data sets',
 'UnderlyingGDPbyIndustry': 'Underlying GDP by Industry',
 'APIDatasetMetaData': 'Metadata about other API datasets'}

In [37]:
res = bea.data('underlying', tablename='u70205s', frequency='m', year=['2015', '2016'])
res.data

,ALLO03,AUTO35,AUTO40,BEACPT,BEASMR,CHG03,CHG04,CHG05,CHG07,CIMPS,...,T4,TEMD,TEMF,TEMG,TEMH,UPR00,UPR01,UPR02,UPR03,UPR05
,,,,,,,,,,,,,,,,,,,,,
2015M01,45.83,3726.4,3542.2,4.271,355.9,-3.9,-5.9,-8.1,1.7,107.6,...,30.7,9.520,7.702,1.382,0.435,25276.0,24153.0,28471.0,26743.0,24139.0
2015M02,46.03,3619.7,3481.4,3.847,320.6,-9.9,-7.9,-8.4,-1.3,106.4,...,31.3,9.677,7.759,1.488,0.430,25418.0,24185.0,28797.0,26946.0,24214.0
2015M03,45.77,3924.7,3740.5,4.247,353.9,10.4,10.4,11.5,24.4,112.0,...,38.0,10.112,8.023,1.652,0.436,25437.0,24083.0,29038.0,26982.0,24199.0
2015M04,44.96,3866.2,3537.8,4.247,353.9,0.0,-0.3,0.5,-5.6,107.9,...,36.2,10.177,8.028,1.705,0.444,25238.0,24086.0,28502.0,26641.0,24068.0
2015M05,44.21,4059.2,3579.7,4.488,374.0,5.7,5.9,4.7,-1.9,114.5,...,37.1,10.181,8.046,1.694,0.440,25195.0,24219.0,28085.0,26491.0,24070.0
2015M06,45.05,3940.7,3612.7,4.086,340.5,-9.0,-9.1,-8.7,-0.2,106.8,...,43.5,10.215,7.935,1.791,0.489,25058.0,24150.0,27690.0,26433.0,23941.0
2015M07,43.19,4110.8,3460.2,4.741,395.1,16.0,16.3,10.7,-16.5,111.6,...,40.6,10.622,8.294,1.838,0.491,25066.0,24175.0,27831.0,26279.0,24044.0
2015M08,45.09,3958.2,3641.9,4.177,348.1,-11.9,-12.1,-5.6,15.6,108.7,...,39.0,10.665,8.317,1.897,0.451,25168.0,23974.0,28716.0,26583.0,24035.0
2015M09,43.85,3954.2,3442.5,4.126,343.8,-1.2,-1.3,-2.7,0.8,110.4,...,37.5,10.746,8.465,1.833,0.447,25148.0,24112.0,28188.0,26470.0,24094.0


In [38]:
import beaapi


In [39]:
list_of_sets = beaapi.get_data_set_list(api_key)
display(list_of_sets)  # Note the last dataset is only for speeding up metadata queries


,DatasetName,DatasetDescription
0,NIPA,Standard NIPA tables
1,NIUnderlyingDetail,Standard NI underlying detail tables
2,MNE,Multinational Enterprises
3,FixedAssets,Standard Fixed Assets tables
4,ITA,International Transactions Accounts
5,IIP,International Investment Position
6,InputOutput,Input-Output Data
7,IntlServTrade,International Services Trade
8,IntlServSTA,International Services Supplied Through Affili...
9,GDPbyIndustry,GDP by Industry


In [72]:
bea_tbl = beaapi.get_data(api_key, datasetname='NIPa', TableName='U50705AM1', Frequency='M', Year='ALL')
display(bea_tbl)

,TableName,SeriesCode,LineNumber,LineDescription,TimePeriod,METRIC_NAME,CL_UNIT,UNIT_MULT,DataValue,NoteRef
0,U50705AM1,A015RC,1,Nonfarm,1967M01,Current Dollars,Level,6,22068,U50705AM1
1,U50705AM1,A015RC,1,Nonfarm,1967M02,Current Dollars,Level,6,9288,U50705AM1
2,U50705AM1,A015RC,1,Nonfarm,1967M03,Current Dollars,Level,6,9672,U50705AM1
3,U50705AM1,A015RC,1,Nonfarm,1967M04,Current Dollars,Level,6,7368,U50705AM1
4,U50705AM1,A015RC,1,Nonfarm,1967M05,Current Dollars,Level,6,5532,U50705AM1
...,...,...,...,...,...,...,...,...,...,...
7807,U50705AM1,A637RC,21,Nondurable goods,1997M08,Current Dollars,Level,6,4670,U50705AM1
7808,U50705AM1,A637RC,21,Nondurable goods,1997M09,Current Dollars,Level,6,4673,U50705AM1
7809,U50705AM1,A637RC,21,Nondurable goods,1997M10,Current Dollars,Level,6,2986,U50705AM1
7810,U50705AM1,A637RC,21,Nondurable goods,1997M11,Current Dollars,Level,6,3066,U50705AM1


In [74]:
bea_tbl.LineDescription.unique()

<StringArray>
[              'Nonfarm',         'Manufacturing',         'Durable goods',
      'Nondurable goods',       'Wholesale trade',    'Merchant wholesale',
 'Nonmerchant wholesale',          'Retail trade', 'Motor vehicle dealers',
                 'Other']
Length: 10, dtype: string